In [18]:
import requests
import shutil
import gzip
import bz2
import os
import re
from bs4 import BeautifulSoup

code for text dataset

In [142]:
url = 'https://dumps.wikimedia.org/enwiki/'
res = requests.get(url)
content = str(res.text)
html_soup = BeautifulSoup(content, 'html.parser')

#print(html_soup)

In [126]:
containers = html_soup.find_all('pre')
links = containers[0].findAll('a', href=True, text=True)
link_list = []
for link in links:
    link_list.append(link['href'].replace('/', '')[:6])
    
# currently available text datasets
available_month = sorted(set(link_list[1:-1]), reverse=True)
print(available_month)

['202003', '202002', '202001', '201912']


In [125]:
# example input1
input_date = '202001'

# default: the first dataset of the month as we'll update the dataset in monthly basis
file_page = url+input_date+'01/'

res_page = requests.get(file_page)
content_page = str(res_page.text)
html_page = BeautifulSoup(content_page, 'html.parser')

In [127]:
container_link = html_page.find_all('li', class_='file')
file_list = []
for link in container_link:
    if ('multistream' in link.text.split(' ')[0]) and ('.xml' in link.text.split(' ')[0]):
        file_list.append(link.text.split(' ')[0])

available_files = file_list[1:]
print(len(available_files))

58


In [134]:
# example input2
input_range = (0, 2)
start, end = input_range

print(start, end)

0 2


In [141]:
def download_file(filename):
    local_filename = filename.split('/')[-1]
    print('Started downloading: ', local_filename)
    print('It may take up to 5 minutes to complete the process.')
    with requests.get(filename, stream=True) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

    return local_filename

In [140]:
for files in available_files[start:end]:
    filename = file_page+files
    local_file = download_file(filename)
    print('Finished downloading: ', local_file, ' in Datasets folder')
    print('Now decompressing the file...')

    new_file_name = '-'.join(local_file.split('-')[:-1])
    comp_file = bz2.BZ2File(local_file, 'rb')
    read_file = comp_file.read()
    comp_file.close()
    f =  open(new_file_name, "wb")
    f.write(read_file)
    f.close()

    shutil.move(new_file_name, 'Datasets/'+new_file_name)
    os.unlink(local_file)

    print('Finished decompressing the file.')


Started downloading:  enwiki-20200101-pages-articles-multistream1.xml-p10p30302.bz2
Finished downloading:  enwiki-20200101-pages-articles-multistream1.xml-p10p30302.bz2  in Datasets folder
Now decompressing the file...
Finished decompressing the file.
Started downloading:  enwiki-20200101-pages-articles-multistream2.xml-p30304p88444.bz2
Finished downloading:  enwiki-20200101-pages-articles-multistream2.xml-p30304p88444.bz2  in Datasets folder
Now decompressing the file...
Finished decompressing the file.


In [143]:
# test small file
#filename = 'https://dumps.wikimedia.org/enwiki/20200101/enwiki-20200101-pages-articles-multistream-index1.txt-p10p30302.bz2'
# test large file
#filename = 'https://dumps.wikimedia.org/enwiki/20200101/enwiki-20200101-pages-articles-multistream1.xml-p10p30302.bz2'


code for clickstream dataset

In [31]:
url = 'https://dumps.wikimedia.org/other/clickstream/'
res = requests.get(url)
content = str(res.text)
html_soup = BeautifulSoup(content, 'html.parser')

#print(html_soup)

In [5]:
containers = html_soup.find_all('pre')
links = containers[0].findAll('a', href=True, text=True)
link_list = []
for link in links:
    link_list.append(link['href'].replace('/', ''))
    

# currently available text datasets
available_month = sorted(set(link_list[1:-1]), reverse=True)
print(available_month)

['2020-01', '2019-12', '2019-11', '2019-10', '2019-09', '2019-08', '2019-07', '2019-06', '2019-05', '2019-04', '2019-03', '2019-02', '2019-01', '2018-12', '2018-11', '2018-10', '2018-09', '2018-08', '2018-07', '2018-06', '2018-05', '2018-04', '2018-03', '2018-02', '2018-01', '2017-12', '2017-11']


In [27]:
# example input1
input_date = '2020-01'

file_page = url+input_date+'/'

res_page = requests.get(file_page)
content_page = str(res_page.text)
html_page = BeautifulSoup(content_page, 'html.parser')

In [35]:
test_date = '202001'
print(test_date[:4]+'-'+test_date[4:])

2020-01


In [28]:
print(file_page)

https://dumps.wikimedia.org/other/clickstream/2020-01/


In [24]:
container_link = html_page.find_all('a', href=True, text=True)
en_file = ''
for link in container_link:
    if ('-enwiki-' in link.text):
        en_file = link.text

print(en_file)

clickstream-enwiki-2020-01.tsv.gz


In [30]:
def download_file(filename):
    local_filename = filename.split('/')[-1]
    print('Started downloading: ', local_filename)
    print('It may take up to 10 minutes to complete the process.')
    with requests.get(filename, stream=True) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

    return local_filename

In [29]:
filename = file_page+en_file
local_file = download_file(filename)
print('Finished downloading: ', local_file, ' in Datasets folder')
print('Now decompressing the file...')

comp_file = gzip.open(local_file, 'rb')
read_file = comp_file.read()
comp_file.close()

new_file_name = en_file.replace('.gz', '')
f =  open(new_file_name, 'wb')
f.write(read_file)
f.close()

shutil.move(new_file_name, 'Datasets/'+new_file_name)
os.unlink(local_file)

print('Finished decompressing the file.')


Started downloading:  clickstream-enwiki-2020-01.tsv.gz
It may take up to 15 minutes to complete the process.
Finished downloading:  clickstream-enwiki-2020-01.tsv.gz  in Datasets folder
Now decompressing the file...
Finished decompressing the file.
